In [296]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [297]:
peliculas = [f for f in listdir("./data") if isfile(join("./data", f))]

In [298]:
for i in peliculas:
    movie = "./data/" + i
    f = open(movie, "r")

In [299]:
key_words = ['Directed','Produced','Written',
             'Based','Starring',
            'Starring', 'Music',  'Cinematography',
             'Edited', 'Production', 'Distributed',
             'Release', 'Running', 'Country',
             'Language', 'Budget', 'Box office', 'Screenplay']

In [300]:
clean_movies = []

In [301]:
def clean_movie(key_movie):
    movie = "./data/" + peliculas[key_movie]
    
    a_file = open(movie, "r")
    list_of_lists = [(line.strip()) for line in a_file]
    a_file.close()
    
    df = pd.DataFrame({'pelicula':list_of_lists})
    df['word'] = np.nan
    
    for key, row in df.iterrows():
        df['titulo'] = str(df.loc[0,'pelicula'])
        for j in range(len(key_words)):
            if key_words[j] in row['pelicula']:
                df.loc[key,'word'] = key_words[j]
                #row['inicio'] = 1
        if (row['pelicula'] in ["Theatrical poster","Theatrical release poster","Teaser poster"]):
            df = df.drop(key)
            
    df = df.reset_index().drop('index', axis=1)
    
    for word in key_words:
        for key, row in df.iterrows():   
            if key > 1:
                condition2 = (str(row['word']) == 'nan') & (df.iloc[key-1]["word"] == word)
                #condition3 = (str(row['word']) == 'nan') & (str(df.iloc[key-1]["word"]) == "nan")
                if  (row['word'] == word) | (condition2):
                    df.loc[key,'word'] = word
    
    return df

In [302]:
clean_movies = [clean_movie(i) for i in range(len(peliculas))]

In [303]:
complete_data = pd.concat(clean_movies)

In [304]:
titulos = complete_data.titulo.unique().tolist()

In [305]:
aiura = []

In [306]:
for title in titulos:
    auxiliar = complete_data[complete_data['titulo'] == title]
    
    for i in key_words:
        j = i + " by"
        p = i + " on"
        k = i + " date"
        auxiliar['pelicula'] = auxiliar['pelicula'].str.replace(j,"")
        auxiliar['pelicula'] = auxiliar['pelicula'].str.replace(p,"")
        auxiliar['pelicula'] = auxiliar['pelicula'].str.replace(k,"")
        auxiliar['pelicula'] = auxiliar['pelicula'].str.replace("Running time","")
        auxiliar['pelicula'] = auxiliar['pelicula'].str.replace(i,"")
    
    auxiliar = auxiliar[1:]
    
    auxiliar['pelicula'].replace('', np.nan, inplace=True)
    
    auxiliar = auxiliar[~auxiliar['pelicula'].isna()]
    
    auxiliar2 = pd.DataFrame()
    
    for w in key_words:
        lista = ""
        for x in auxiliar[auxiliar["word"] == w].pelicula.unique().tolist():
            if len(auxiliar[auxiliar["word"] == w].pelicula.unique().tolist()) == 1:
                lista = str(x) 
            else: 
                lista = str(x)+", "+ lista
        auxiliar2[w] = [lista]
    
    auxiliar2['titulo'] = title
    aiura.append(auxiliar2)

/home/ant/anaconda3/envs/orax_report/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ant/anaconda3/envs/orax_report/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ant/anaconda3/envs/orax_report/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [307]:
movies_data = pd.concat(aiura)

In [308]:
movies_data[movies_data['titulo'] != 'Danger Theatre'].to_csv("movie_data.csv", index=False)

In [309]:
movies_data

,Directed,Produced,Written,Based,Starring,Music,Cinematography,Edited,Production,Distributed,Release,Running,Country,Language,Budget,Box office,Screenplay,titulo
0,Ciro Guerra,Cristina Gallego,"Jacques Toulemonde Vidal, Ciro Guerra,",Theodor Koch-Grunberg and Richard Evans Schul...,"Antonio Bolívar, Nilbio Torres, Jan Bijvoet,",Nascuy Linares,David Gallego,Etienne Boussac,"Nortesur Producciones, MC Producciones, Dago G...",Diaphana Films,"[1], 21 May 2015 (Colombia), 15 May 2015 (Cann...",125 minutes,"Argentina, Venezuela, Colombia,","English, Latin, Catalan, German, Portuguese, S...",$1.4 million[2],$3.4 million[3],,Embrace of the Serpent
0,Scott Cooper,"John Lesher, Ken Kao, Scott Cooper,",,,"Ben Foster, Rory Cochrane, Adam Beach, Jesse P...",Max Richter,Masanobu Takayanagi,Tom Cross,"Bloom Media, Le Grisbi s, Waypoint Entertainme...",Entertainment Studios Motion Pictures,"December 22, 2017 (United States), September 2...",133 minutes[1],United States,"Cheyenne, English,",$39 million[2][3][4],$35.7 million[5],"Story by Donald E. Stewart, Scott Cooper,",Hostiles
0,Mel Gibson,"Bruce Davey, Mel Gibson,","Farhad Safinia, Mel Gibson,",,"Israel Contreras, Ricardo Diaz Mendoza, Ammel ...",James Horner,Dean Semler,John Wright,"Icon s, Touchstone Pictures, company,",Buena Vista Pictures,"December 8, 2006",138 minutes[1],United States,Yucatec Mayan,$40 million[2],$120.7 million[2],,Apocalypto
0,,,,,,,,,company(s) Universal Television,,"August 22, 1993, Original release July 11 –, O...",30 minutes,"No. of episodes 7, No. of seasons 1, Original ...",,,,,Danger Theatre
0,Kevin Costner,"Kevin Costner, Jim Wilson,",,"by Michael Blake, Dances with Wolves,","Rodney Grant, Graham Greene, Mary McDonnell, K...",John Barry,Dean Semler,Neil Travis,"Tig s, companies,",Orion Pictures,"November 9, 1990 (United States), October 19, ...",181 minutes[1],United States,"Pawnee, Lakota, English,",$22 million[2],$424.2 million[2],Michael Blake,Dances with Wolves
0,Lisandro Alonso,"Helle Ulsteen, Jaime Romandia, Sylvie Pialat, ...","Lisandro Alonso, Fabian Casas,",,Viggo Mortensen,Viggo Mortensen,Timo Salminen,"Natalia López, Gonzalo del Val,",,NDM,"27 November 2014 (Argentina), 18 May 2014 (Can...",110 minutes[1],"United States, Netherlands, Mexico, France, Ar...","French, Spanish, Danish,",,,,Jauja
0,David Oelhoffen,"Matthew Gledhill, Marc Du Pontavice,",,"by Albert Camus, The Guest,","Reda Kateb, Viggo Mortensen,","Warren Ellis, Nick Cave,",Guillaume Deffontaines,Juliette Welfling,,Pathé,"14 January 2015 (France), 31 August 2014 (Veni...",110 minutes,France,"Spanish, Arabic, French,",$5.4 million,"$835,031[1]",David Oelhoffen,Far from Men
0,Terrence Malick,Sarah Green,Terrence Malick,,"Christian Bale, Christopher Plummer, Q'orianka...",James Horner,Emmanuel Lubezki,"Mark Yoshikawa, Saar Klein, Hank Corwin, Richa...","Sunflower s, Sarah Green Film, First Foot Film...",New Line Cinema,"January 27, 2006 (United Kingdom), December 25...","(2006 wide release), 136 minutes[2], (2005 lim...","United States, United Kingdom,","Algonquian, English,",$30 million[3],$30.5 million[3],,The New World
0,Jim Jarmusch,Demetra J. MacBride,Jim Jarmusch,,"Robert Mitchum, Alfred Molina, John Hurt, Gabr...",Neil Young,Robby Müller,Jay Rabinowitz,,Miramax Films,"May 10, 1996, May 26, 1995 (Cannes),",120 minutes[1],United States,English,$9 million[2],$1 million [3],,Dead Man
0,Alejandro G. Iñárritu,"James W. Skotchdopole, Keith Redmon, Mary Pare...",,"by Michael Punke, The Revenant,","Will Poulter, Domhnall Gleeson, Tom Hardy, Leo...","Alva Noto, Ryuichi Sakamoto,",Emmanuel Lubezki,Stephen Mirrione,"Appian Way s, M s, Anonymous Content, New Rege...",20th Century Fox,"December 25, 2015 (United States), December 16...",156 minutes[1],United States[2],English,$135 million[3],$533 million[4],"Alejandro G. Iñárritu, Mark L. Smith,",The Revenant
